In [1]:
from google.genai import types # type: ignore
from google import genai
import pandas as pd
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
from app.GeminiClient import GeminiClient
import time
from app.MariaDBClient import MariaDBClient
from app.MarketScraper import MarketScraper
import pandas as pd
import asyncio

In [2]:
# https://www.listendata.com/2024/05/how-to-use-gemini-in-python.html
# Create API KEY: https://aistudio.google.com/app/u/0/apikey?pli=1

In [4]:
mariadbclient = MariaDBClient()
supermercados = mariadbclient.get_active_tables()
market_scraper = MarketScraper()
total_items = []
total_stats = []
for supermercado in supermercados:
    data = market_scraper.fetch_data(supermercado)
    items = market_scraper.get_items(data)
    total_items.append(items)
    stas = market_scraper.get_stats(data)
    total_stats.append(stas)

total_items_df = pd.concat(total_items, ignore_index=True)
total_stats_df = pd.concat(total_stats, ignore_index=True)


In [5]:
new_items_df = total_items_df

In [6]:
old_items_df = mariadbclient.get_products()

In [7]:
new_only_df = new_items_df[~new_items_df['id_producto'].isin(old_items_df['id_producto'])] # type: ignore
common_df = new_items_df[new_items_df['id_producto'].isin(old_items_df['id_producto'])]
old_only_df = old_items_df[~old_items_df['id_producto'].isin(new_items_df['id_producto'])]
print(f"Nuevos productos: {new_only_df.shape[0]}")
print(f"Productos ya existentes: {common_df.shape[0]}")
print(f"Productos ya no existentes: {old_only_df.shape[0]}")

Nuevos productos: 2
Productos ya existentes: 1205
Productos ya no existentes: 0


In [8]:
mariadbclient.delete_products(old_only_df)
print("Se eliminaron ",len(old_only_df), " productos")
mariadbclient.update_products(common_df)
print("Se actualizaron ",len(common_df), " productos")

Se eliminaron  0  productos
Se actualizaron  1205  productos


In [10]:
old_productos_to_update_subgroups_df = mariadbclient.get_products_by_groups(new_only_df)
new_productos_to_update_subgroups_df = new_only_df[['id_producto', 'categoria', 'marca', 'descripcion', 'volumen', 'id_supermercado']]
new_productos_to_update_subgroups_df = new_productos_to_update_subgroups_df.rename(columns={'id_supermercado': 'supermercado'})
new_productos_to_update_subgroups_df['supermercado'] = new_productos_to_update_subgroups_df['supermercado'].map({
    1:"Atomo",
    2:"Blowmax",
    3:"ModoMarket",
    4:"Segal",
    5:"Supera"
})
all_productos_to_upsert_subgroups_df = pd.concat([new_productos_to_update_subgroups_df, old_productos_to_update_subgroups_df], ignore_index=True)
all_productos_to_upsert_subgroups_df['producto_id'] = all_productos_to_upsert_subgroups_df['id_producto'].astype(str) + " - " + all_productos_to_upsert_subgroups_df['categoria'] + " - " + all_productos_to_upsert_subgroups_df['marca'] + " - " + all_productos_to_upsert_subgroups_df['descripcion'] + " - " + all_productos_to_upsert_subgroups_df['volumen'] + " - " + all_productos_to_upsert_subgroups_df['supermercado']


In [11]:
print("Hay ", len(old_productos_to_update_subgroups_df), " productos que comparten subgrupo con los productos nuevos")
print("Hay ", len(new_productos_to_update_subgroups_df), " productos nuevos que hay que asignar subgrupo")
print("Hay ", len(all_productos_to_upsert_subgroups_df), " productos que hay que actualizar su subgrupo")

Hay  3  productos que comparten subgrupo con los productos nuevos
Hay  2  productos nuevos que hay que asignar subgrupo
Hay  5  productos que hay que actualizar su subgrupo


In [12]:
grouped = all_productos_to_upsert_subgroups_df.groupby(['categoria', 'marca', 'volumen'])
listas_productos = [group['producto_id'].tolist() for _, group in grouped]

geminiclient = GeminiClient()
geminiclient.api_keys

total_elementos = sum(len(sublista) for sublista in listas_productos)
all_responses = []

indice = 0
for lista_productos in listas_productos:
    with open('agrupacion.txt', 'a') as file:
        prompt = listas_productos[indice]
        try:
            if len(prompt) > 1:
                respuesta = geminiclient.ask(prompt)
                time.sleep(4.1)
            else:
                respuesta = [prompt]
            all_responses.append(respuesta)
        except Exception as e:
            print(f"Error al procesar el prompt: {prompt}")
            print(f"Error: {e}")
    indice += 1

In [13]:
print("Gemini tiene que agrupar ",total_elementos, " productos")
total_agrupados = sum(len(subsublist) for sublist in all_responses for subsublist in sublist)
print("Gemini agrupo ",total_agrupados, " productos")


Gemini tiene que agrupar  5  productos
Gemini agrupo  5  productos


In [14]:
all_responses

[[['199299224 - Gaseosas - COCA COLA - - 354 - Atomo',
   '3420491408 - Gaseosas - COCA COLA - - 354 - Blowmax',
   '3022578047 - Gaseosas - COCA COLA - - 354 - Modo Market'],
  ['1503644882 - Gaseosas - COCA COLA - SIN AZUCAR - 354 - Atomo',
   '2841869928 - Gaseosas - COCA COLA - SIN AZUCAR - 354 - Blowmax']]]

In [15]:
olds_ids = set(old_productos_to_update_subgroups_df['id_producto'])
news_ids = set(new_items_df['id_producto'])

# Preparamos las estructuras de resultados
updates = []
descartados = []

# Creamos un diccionario para mapear rápidamente los IDs a sus índices en new_items_df
new_items_index = {id_producto: idx for idx, id_producto in enumerate(new_items_df['id_producto'])}

# Procesamos todas las respuestas
for respuesta in all_responses:
    for j, subgrupo in enumerate(respuesta):
        ids_productos = [item.partition(' - ')[0].strip() for item in subgrupo]  # .strip() para eliminar espacios
        
        for id_producto_str in ids_productos:
  
                id_producto = int(id_producto_str)
                
                if id_producto in olds_ids:
                    updates.append({ 
                        "id_producto": id_producto,
                        "id_subgrupo": j
                    })
                elif id_producto in news_ids:
                    idx = new_items_index[id_producto]
                    new_only_df.at[idx, 'id_subgrupo'] = j
                else:
                    descartados.append(id_producto_str)


In [16]:
new_only_df

,__ingestion_timestamp,nombre_crudo,id_producto,categoria,nombre,marca,descripcion,volumen,precio,id_supermercado,id_grupo,id_subgrupo,url
230,2025-04-29 22:35:17.991353,GASEOSAS COCA COLA LATA 354 CC.,199299224,Gaseosas,COCA COLA 354,COCA COLA,,354,950.0,1,1792792882,0.0,https://atomoconviene.com/atomo-ecommerce/cola...
231,2025-04-29 22:35:17.991353,GASEOSA COCA COLA ZERO LATA 354 CC.,1503644882,Gaseosas,COCA COLA SIN AZUCAR 354,COCA COLA,SIN AZUCAR,354,950.0,1,1792792882,1.0,https://atomoconviene.com/atomo-ecommerce/gase...


In [17]:
updates

[{'id_producto': 3420491408, 'id_subgrupo': 0},
 {'id_producto': 3022578047, 'id_subgrupo': 0},
 {'id_producto': 2841869928, 'id_subgrupo': 1}]

In [18]:
descartados

[]

In [19]:
mariadbclient.insert_into_productos(new_only_df)
mariadbclient.update_product_subgroups(updates)